In [8]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from transformers import TFAutoModel, AutoTokenizer

In [2]:
df = pd.read_csv("../data/dataset_csv/dataset_pidana_umum.csv")

df.head()

,amar,amar_lainnya,id,klasifikasi,lama_hukuman,lembaga_peradilan,provinsi,status,sub_klasifikasi,url,...,identitas,riwayat_penahanan,riwayat_perkara,riwayat_tuntutan,riwayat_dakwaan,fakta,amar_putusan,penutup,fakta_hukum,pertimbangan_hukum
0,pidana,hukum,00035681c8d944203f25d2e8215ae2bf,pidana-umum,210,pn-kudus,jateng,berkekuatan-hukum-tetap,pemalsuan,https://putusan3.mahkamahagung.go.id/direktori...,...,nama lengkap eny sulistiyaningsih binti mashad...,terdakwa ditahan dengan jenis tahanan rutan se...,pengadilan negeri tersebut\nsetelah membaca be...,setelah mendengar tuntutan requsitoir penuntut...,menimbang bahwa terdakwa diajukan di persidang...,menimbang bahwa selanjutnya untuk membuktikan ...,mengadili 1 menyatakan terdakwa eny sulistiyan...,demikian diputuskan dalam rapat permusyawarata...,NaN,NaN
1,pidana,hukum,000399ce26773e18695ce14f519cb9e6,pidana-umum,720,pn-demak,jateng,berkekuatan-hukum-tetap,pencurian,https://putusan3.mahkamahagung.go.id/direktori...,...,nama lengkap ali maftuhin bin nur salim tempat...,terdakwa ditahan di rumah tahanan negara berda...,pengadilan negeri tersebut\nsetelah membaca\np...,setelah mendengar surat tuntutan pidana requis...,menimbang bahwa terdakwa didakwa oleh penuntut...,menimbang bahwa untuk menguatkan dakwaan terse...,mengadili 1 menyatakan terdakwa ali maftuhin b...,demikianlah diputuskan dalam rapat permusyawar...,menimbang bahwa berdasarkan keterangan saksi s...,NaN
2,pidana,jatuh-pidana-oleh-karena-itu-kepada-dakwa-ir-b...,0006582ad67cd9bd1ddf4261a09bf382,pidana-umum,120,pn-kediri,jatim,berkekuatan-hukum-tetap,kejahatan-terhadap-keamanan-negara,https://putusan3.mahkamahagung.go.id/direktori...,...,nama lengkap ir bambang sasongko bin r soewarn...,terdakwa tidak ditahan,terdakwa didampingi oleh penasehat hukumnya ya...,telah mendengar pembacaan tuntutan pidana oleh...,menimbang bahwa terdakwa diajukan di persidang...,menimbang bahwa selanjutnya dipersidangan tela...,mengadili\n1 menyatakan terdakwa ir bambang sa...,demikian diputuskan dalam rapat musyawarah maj...,NaN,NaN
3,pidana,hukum,00122b1be15a10ad474bb3b7ec0dea73,pidana-umum,90,pn-sampang,jatim,berkekuatan-hukum-tetap,penghinaan,https://putusan3.mahkamahagung.go.id/direktori...,...,nama lengkap ahmad al pak saki tempat lahir sa...,NaN,terdakwa dipersidangan tidak didampingi oleh p...,telah mendengar tuntutan pidana dari penuntut ...,menimbang bahwa berdasarkan catatan penuntut u...,menimbang bahwa dalam persidangan telah dideng...,mengadili 1 menyatakan terdakwa ahmad al pak s...,demikian diputuskan pada hari senin tanggal 5 ...,menimbang bahwa berdasarkan keterangan saksi s...,NaN
4,pidana,hukum,00136d1554e18c63256deac42aad0c58,pidana-umum,210,pn-cirebon,jabar,berkekuatan-hukum-tetap,pencurian,https://putusan3.mahkamahagung.go.id/direktori...,...,1 nama lengkap muhamad rizki als rizki bin edi...,terdakwa ditangkap pada tanggal juli 2019\nter...,terdakwa tidak didampingi penasihat hukum\npen...,setelah mendengar pembacaan tuntutan pidana ya...,menimbang bahwa terdakwa diajukan ke persidang...,menimbang bahwa untuk membuktikan dakwaannya p...,mengadili\n1 menyatakan terdakwa muhamad rizki...,demikian diputuskan dalam sidang permusyawarat...,menimbang bahwa berdasarkan alat bukti dan bar...,menimbang bahwa selanjutnya majelis hakim akan...


In [3]:
# 1. Pilih fitur dan label
features = df['riwayat_dakwaan']  # Teks yang digunakan untuk prediksi
labels = df['sub_klasifikasi']    # Kategori pidana

# Drop nilai kosong dan pastikan input valid
df = df.dropna(subset=['riwayat_dakwaan', 'sub_klasifikasi'])
df['riwayat_dakwaan'] = df['riwayat_dakwaan'].astype(str)

# Encode Label
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)
num_classes = len(label_encoder.classes_)

# Split data menjadi train dan test
X_train, X_test, y_train, y_test = train_test_split(features, encoded_labels, test_size=0.2, random_state=42)


In [4]:
# 2. Load IndoBERT dan Tokenizer
model_name = "indobenchmark/indobert-base-p2"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Tokenisasi teks
def tokenize_texts(texts, tokenizer, max_length=200):
    # Validasi input: konversi ke string dan filter nilai kosong
    texts = [str(text) for text in texts if isinstance(text, str) and text.strip() != ""]
    return tokenizer(
        texts,
        max_length=max_length,
        padding="max_length",
        truncation=True,
        return_tensors="tf"
    )

X_train = X_train.astype(str)
X_test = X_test.astype(str)

train_encodings = tokenize_texts(X_train, tokenizer)
test_encodings = tokenize_texts(X_test, tokenizer)

In [10]:
# 3. Functional API untuk Model
input_ids = tf.keras.layers.Input(shape=(200,), dtype=tf.int32, name="input_ids")
attention_mask = tf.keras.layers.Input(shape=(200,), dtype=tf.int32, name="attention_mask")

pretrained_bert = TFAutoModel.from_pretrained(model_name)
bert_outputs = pretrained_bert(input_ids, attention_mask=attention_mask)
cls_token = bert_outputs.last_hidden_state[:, 0, :]  # Ambil representasi token [CLS]

dense_layer = tf.keras.layers.Dense(128, activation='relu')(cls_token)
dropout_layer = tf.keras.layers.Dropout(0.3)(dense_layer)
output_layer = tf.keras.layers.Dense(num_classes, activation='softmax')(dropout_layer)

model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=output_layer)

c:\Users\ASUS\.virtualenvs\Machine_Learning-95sCrP9Y\lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ASUS\.cache\huggingface\hub\models--indobenchmark--indobert-base-p2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


MemoryError: Unable to allocate 146. MiB for an array with shape (50000, 768) and data type float32

In [ ]:
# Compile Model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
# Train Model
history = model.fit(
    x={
        "input_ids": train_encodings['input_ids'],
        "attention_mask": train_encodings['attention_mask']
    },
    y=y_train,
    validation_data=(
        {
            "input_ids": test_encodings['input_ids'],
            "attention_mask": test_encodings['attention_mask']
        },
        y_test
    ),
    epochs=5,
    batch_size=16
)

In [ ]:
# Evaluasi Model
loss, accuracy = model.evaluate(
    x={
        "input_ids": test_encodings['input_ids'],
        "attention_mask": test_encodings['attention_mask']
    },
    y=y_test
)
print(f"Accuracy: {accuracy}")

In [ ]:
# Prediksi pada Data Baru
predictions = model.predict({
    "input_ids": test_encodings['input_ids'][:5],
    "attention_mask": test_encodings['attention_mask'][:5]
})
predicted_classes = label_encoder.inverse_transform(predictions.argmax(axis=1))
print("Predicted Categories:", predicted_classes)

In [ ]:
example_text = "Menimbang, bahwa terdakwa..."
tokenized_example = tokenizer(example_text, max_length=200, padding="max_length", truncation=True)
print(tokenized_example)